#Install Dependencies

In [1]:
!git clone https://github.com/snap-stanford/snap
import os 
os.chdir("snap/examples/node2vec/")  

Cloning into 'snap'...
remote: Enumerating objects: 12838, done.
remote: Total 12838 (delta 0), reused 0 (delta 0), pack-reused 12838
Receiving objects: 100% (12838/12838), 157.05 MiB | 27.02 MiB/s, done.
Resolving deltas: 100% (9027/9027), done.


In [2]:
!make

make -C ../../snap-core
make[1]: Entering directory '/content/snap/snap-core'
g++ -c -std=c++98 -Wall -O3 -DNDEBUG -fopenmp Snap.cpp -I../glib-core
In file included from ../glib-core/base.h:182:0,
                 from Snap.h:9,
                 from Snap.cpp:4:
../glib-core/ds.h: In member function ‘bool TTuple<TVal, NVals>::operator==(const TTuple<TVal, NVals>&) const’:
../glib-core/ds.h:290:5: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
     for (int i=0; i<Len(); i++) if(ValV[i]!=Tup[i]){return false;} return true; }
     ^~~
../glib-core/ds.h:290:68: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
     for (int i=0; i<Len(); i++) if(ValV[i]!=Tup[i]){return false;} return true; }
                                                                    ^~~~~~
In file included from Snap.cpp:8:0:
../glib-core/linalg.cpp: In static member function ‘static void TNumericalStuff::LUSolve(const TFltVV&, const TIntV&

#Download  The Data

In [3]:
!gdown https://drive.google.com/uc?id=13q11ZnhYehDtibrHCvA7kfSkCKcxMpcv

import zipfile
with zipfile.ZipFile("fr-domain-classification.zip","r") as zip_ref:
    zip_ref.extractall("")
    

Downloading...
From: https://drive.google.com/uc?id=13q11ZnhYehDtibrHCvA7kfSkCKcxMpcv
To: /content/snap/examples/node2vec/fr-domain-classification.zip
259MB [00:02, 90.3MB/s]


#Load Dependencies

In [0]:
import numpy as np
import networkx as nx
from random import randint
from gensim.models import Word2Vec
from tqdm import tqdm
import csv
import numpy as np
import os 

# Load The Data

## Load The Undirected Graph


In [0]:
G = nx.read_weighted_edgelist('edgelist.txt', create_using=nx.Graph())
G.add_node("17591")

# Load The Train And Test Node Index

In [0]:
with open("train.csv") as f :
  train_data = f.read().splitlines()
train_hosts = list()

y_train = list()
for row in train_data:
    host, label = row.split(",")
    train_hosts.append(host)
    y_train.append(label.lower())

with open("test.csv", 'r') as f:
    test_hosts = f.read().splitlines()

#Categorical Encoding Using Label Encoding

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
le.classes_
y_train_num = le.transform(y_train)


#Node2Vec

In [0]:
nx.write_edgelist(G, "graph.edgelist")

In [0]:
%%bash
./node2vec -i:graph.edgelist -o:embdings.emb  -e:10 -d:256 -l:400  -r:100


#Save Embedding Matrix

In [0]:
n_dim = 256
embeddings = np.zeros((G.number_of_nodes(), n_dim))
with open("embdings.emb", "r") as movies_file:
    next(movies_file)
    reader = csv.reader(movies_file, delimiter=" ")

    params = []
    for row in reader:
        embeddings[int(row[0]),:] =[float(item) for item in row[1:]]

np.save("embedding_node2_vec",embeddings)      

#Deep Walk

## Generate Random

In [0]:
def random_walk(G, node, walk_length):
  walk = [node]

  for i in range(walk_length):
    neighbors =  list(G.neighbors(walk[i]))
    walk.append(neighbors[randint(0,len(neighbors)-1)])
  walk = [str(node) for node in walk]
  return walk

In [15]:
walks = []
num_walks=20
walk_lenght=500
for i in tqdm(range(num_walks)):  
		nodes = G.nodes()
		nodes = np.random.permutation(nodes)
		for host in train_hosts+test_hosts:
     
			walk = random_walk(G, host, walk_lenght)
			walks.append(walk)




100%|██████████| 20/20 [00:10<00:00,  1.95it/s]


## Train Word2Vec

In [16]:
model = Word2Vec(size=256, window=10, min_count=0, sg=1, workers=8)
model.build_vocab(walks)
model.train(walks, total_examples=model.corpus_count, epochs=5)

(5443883, 5638500)

#Save Embedding Matrix

In [0]:
embeddings = np.empty(shape=(len( train_hosts+test_hosts), 256))
for idx,node in enumerate( train_hosts+test_hosts):
        embeddings[idx,:] = model.wv[node]

In [0]:
np.save("embedding_deep_walk",embeddings) 

#BioNEV(Biomedical Network Embedding Evaluation)

In [0]:
!pip install git+https://github.com/xiangyue9607/BioNEV.git

## SDNE

In [0]:
!bionev --input edgelist.txt --output SDNE.txt    --method SDNE  --weighted True

In [0]:
with open('SDNE.txt') as f:
    lines = f.readlines()

embeddings=np.zeros((G.number_of_nodes(),128))
for line in lines[1:]:
  tmp=line.replace("\n","").split(" ")
  vec=[]
  for u in tmp[1:]:
    vec.append(float(u))
  embeddings[int(tmp[0]),:]=vec

np.save("embeddings_SDNE.npy",embeddings)

## LINE

In [0]:
!bionev --dimensions 256 --input edgelist.txt --output LINE.txt  --method LINE --order 3 --weighted True

In [0]:
with open('LINE.txt') as f:
    lines = f.readlines()

embeddings=np.zeros((G.number_of_nodes(),256))
for line in lines[1:]:
  tmp=line.replace("\n","").split(" ")
  vec=[]
  for u in tmp[1:]:
    vec.append(float(u))
  embeddings[int(tmp[0]),:]=vec


np.save("embeddings_LINE.npy",embeddings)

#Singular Value Decomposition

In [22]:
!bionev --input edgelist.txt \
       --output SVD.txt \
       --method SVD \
        --dimensions 256 \
       --weighted True

######################################################################
Embedding Method: SVD, Evaluation Task: none
######################################################################
Embedding Learning Time: 28.34 s


In [0]:
with open('SVD.txt') as f:
    lines = f.readlines()

embeddings=np.zeros((G.number_of_nodes(),256))
for line in lines[1:]:
  tmp=line.replace("\n","").split(" ")
  vec=[]
  for u in tmp[1:]:
    vec.append(float(u))
  embeddings[int(tmp[0]),:]=vec


np.save("embeddings_SVD.npy",embeddings)